# Controlling Burgers' Equation with Reinforcement Learning
This notebook will demonstrate how reinforcement learning can be used to control Burgers' equation, a nonlinear PDE. The approach uses the reinforcement learning framework [stable_baselines3](https://github.com/DLR-RM/stable-baselines3) and the differentiable PDE solver [Φ<sub>Flow</sub>](https://github.com/tum-pbs/PhiFlow). As a reinforcement learning algorithm, [PPO](https://arxiv.org/abs/1707.06347v2) was selected.

In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../src'); sys.path.append('../PDE-Control/PhiFlow'); sys.path.append('../PDE-Control/src')
from phi.flow import *
import matplotlib.pyplot as plt
import burgers_plots as bplt
from experiment import BurgersTraining

/home/felix/anaconda3/envs/phiflow/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/felix/anaconda3/envs/phiflow/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/felix/anaconda3/envs/phiflow/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/felix/anaconda3/envs/phiflow/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWarning:

Passing (type, 1) or '1type' as a synony

## Reinforcement Learning Initialization

In [2]:
domain = Domain([32], box=box[0:1]) # 1d grid resolution and physical size
viscosity = 0.003 # viscosity constant for Burgers' equation
step_count = 32 # length of each trajectory
dt = 0.03 # time step size
diffusion_substeps = 1 # how many diffusion steps to perform at each solver step

n_envs = 10 # On how many environments to train in parallel, load balancing
final_reward_factor = step_count # How hard to punish the agent for not reaching the goal if that is the case
steps_per_rollout = step_count * 10 # How many steps to collect per environment between agent updates
training_timesteps = steps_per_rollout * 1000 # How long the actual training should be
n_epochs = 10 # How many epochs to perform during agent update
learning_rate = 1e-4 # Learning rate for agent updates
batch_size = 128 # Batch size for agent updates
test_path = 'forced-burgers-clash' # Path of the used test set for comparison to cfe method
test_range = range(100) # Test samples inside the dataset

To start training, we create a trainer object, which manages the environment and the agent internally. Additionally, a directory for storing models, logs, and hyperparameters is created. This way, training can be continued at any later point using the same configuration. If the model folder specified in `exp_name` already exists, the agent within is loaded. Otherwise, a new agent is created

In [3]:
trainer = BurgersTraining(
    exp_name='../networks/rl-models/ControlBurgersBench',
    domain=domain,
    viscosity=viscosity,
    step_count=step_count,
    dt=dt,
    diffusion_substeps=diffusion_substeps,
    n_envs=n_envs,
    final_reward_factor=final_reward_factor,
    steps_per_rollout=steps_per_rollout,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    batch_size=batch_size,
    test_path=test_path,
    test_range=range(100),
)

Using new running mean for reward
Creating new agent...
Using given reward rms
Using cuda device


Now we are set up to start training the agent. The next line will take quite some time to execute, so grab a coffee or take your dog for a walk or so.

`n_rollouts` denotes the length of the training

`save_freq` specifies the number of epochs after which the stored model is overwritten

In [4]:
trainer.train(n_rollouts=1000, save_freq=50)

Logging to ../networks/rl-models/ControlBurgersBench/tensorboard-log/PPO_1
Storing agent and hyperparameters to disk...


/home/felix/anaconda3/envs/phiflow/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning:

Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.



Forces on test set: 34499.577050
-----------------------------------
| forces             | 1143.5386  |
| rew_unnormalized   | -262437.3  |
| rollout/           |            |
|    ep_len_mean     | 32.0       |
|    ep_rew_mean     | -0.5929396 |
| test set forces    | 3.45e+04   |
| time/              |            |
|    fps             | 664        |
|    iterations      | 1          |
|    time_elapsed    | 4          |
|    total_timesteps | 3200       |
-----------------------------------
Forces on test set: 35935.269890
-----------------------------------------
| forces                  | 1124.006    |
| rew_unnormalized        | -245304.72  |
| rollout/                |             |
|    ep_len_mean          | 32.0        |
|    ep_rew_mean          | 0.13344853  |
| test set forces         | 3.59e+04    |
| time/                   |             |
|    fps                  | 362         |
|    iterations           | 2           |
|    time_elapsed         | 17          |
|   

Forces on test set: 37768.909721
-----------------------------------------
| forces                  | 1143.9934   |
| rew_unnormalized        | -269292.6   |
| rollout/                |             |
|    ep_len_mean          | 32.0        |
|    ep_rew_mean          | -0.19531393 |
| test set forces         | 3.78e+04    |
| time/                   |             |
|    fps                  | 269         |
|    iterations           | 10          |
|    time_elapsed         | 118         |
|    total_timesteps      | 32000       |
| train/                  |             |
|    approx_kl            | 0.006911783 |
|    clip_fraction        | 0.0355      |
|    clip_range           | 0.2         |
|    entropy_loss         | -45.4       |
|    explained_variance   | 0.941       |
|    learning_rate        | 0.0001      |
|    loss                 | 0.123       |
|    n_updates            | 90          |
|    policy_gradient_loss | -0.00897    |
|    std                  | 1           |
|

Forces on test set: 38034.041977
------------------------------------------
| forces                  | 1143.6226    |
| rew_unnormalized        | -258097.95   |
| rollout/                |              |
|    ep_len_mean          | 32.0         |
|    ep_rew_mean          | -0.10245581  |
| test set forces         | 3.8e+04      |
| time/                   |              |
|    fps                  | 240          |
|    iterations           | 18           |
|    time_elapsed         | 239          |
|    total_timesteps      | 57600        |
| train/                  |              |
|    approx_kl            | 0.0048261476 |
|    clip_fraction        | 0.0571       |
|    clip_range           | 0.2          |
|    entropy_loss         | -45.4        |
|    explained_variance   | 0.979        |
|    learning_rate        | 0.0001       |
|    loss                 | 0.0861       |
|    n_updates            | 170          |
|    policy_gradient_loss | -0.0102      |
|    std             

...

Forces on test set: 66006.633545
-----------------------------------------
| forces                  | 942.2854    |
| rew_unnormalized        | -4735.7236  |
| rollout/                |             |
|    ep_len_mean          | 32.0        |
|    ep_rew_mean          | 1.2524091   |
| test set forces         | 6.6e+04     |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 962         |
|    time_elapsed         | 12054       |
|    total_timesteps      | 3078400     |
| train/                  |             |
|    approx_kl            | 0.036437035 |
|    clip_fraction        | 0.36        |
|    clip_range           | 0.2         |
|    entropy_loss         | -35.8       |
|    explained_variance   | 0.943       |
|    learning_rate        | 0.0001      |
|    loss                 | -0.0202     |
|    n_updates            | 9610        |
|    policy_gradient_loss | -0.0228     |
|    std                  | 0.742       |
|

Forces on test set: 67034.364624
-----------------------------------------
| forces                  | 982.74475   |
| rew_unnormalized        | -5660.987   |
| rollout/                |             |
|    ep_len_mean          | 32.0        |
|    ep_rew_mean          | 1.1619529   |
| test set forces         | 6.7e+04     |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 970         |
|    time_elapsed         | 12157       |
|    total_timesteps      | 3104000     |
| train/                  |             |
|    approx_kl            | 0.042912778 |
|    clip_fraction        | 0.345       |
|    clip_range           | 0.2         |
|    entropy_loss         | -35.7       |
|    explained_variance   | 0.953       |
|    learning_rate        | 0.0001      |
|    loss                 | -0.0406     |
|    n_updates            | 9690        |
|    policy_gradient_loss | -0.0257     |
|    std                  | 0.74        |
|

Forces on test set: 68347.049438
-----------------------------------------
| forces                  | 955.6172    |
| rew_unnormalized        | -4863.553   |
| rollout/                |             |
|    ep_len_mean          | 32.0        |
|    ep_rew_mean          | 1.2308255   |
| test set forces         | 6.83e+04    |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 978         |
|    time_elapsed         | 12261       |
|    total_timesteps      | 3129600     |
| train/                  |             |
|    approx_kl            | 0.041225195 |
|    clip_fraction        | 0.38        |
|    clip_range           | 0.2         |
|    entropy_loss         | -35.6       |
|    explained_variance   | 0.945       |
|    learning_rate        | 0.0001      |
|    loss                 | -0.0383     |
|    n_updates            | 9770        |
|    policy_gradient_loss | -0.0208     |
|    std                  | 0.738       |
|

Forces on test set: 68810.727295
-----------------------------------------
| forces                  | 990.0867    |
| rew_unnormalized        | -6608.841   |
| rollout/                |             |
|    ep_len_mean          | 32.0        |
|    ep_rew_mean          | 1.0637099   |
| test set forces         | 6.88e+04    |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 986         |
|    time_elapsed         | 12365       |
|    total_timesteps      | 3155200     |
| train/                  |             |
|    approx_kl            | 0.049877357 |
|    clip_fraction        | 0.379       |
|    clip_range           | 0.2         |
|    entropy_loss         | -35.4       |
|    explained_variance   | 0.951       |
|    learning_rate        | 0.0001      |
|    loss                 | -0.00366    |
|    n_updates            | 9850        |
|    policy_gradient_loss | -0.023      |
|    std                  | 0.734       |
|

Forces on test set: 67934.624512
-----------------------------------------
| forces                  | 936.6354    |
| rew_unnormalized        | -5056.4937  |
| rollout/                |             |
|    ep_len_mean          | 32.0        |
|    ep_rew_mean          | 1.203294    |
| test set forces         | 6.79e+04    |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 994         |
|    time_elapsed         | 12469       |
|    total_timesteps      | 3180800     |
| train/                  |             |
|    approx_kl            | 0.038654134 |
|    clip_fraction        | 0.332       |
|    clip_range           | 0.2         |
|    entropy_loss         | -35.3       |
|    explained_variance   | 0.971       |
|    learning_rate        | 0.0001      |
|    loss                 | -0.00474    |
|    n_updates            | 9930        |
|    policy_gradient_loss | -0.0256     |
|    std                  | 0.731       |
|

Let's see how the reward evolved during training

In [ ]:
trainer.plot()

Run the cell below to open the tensorboard logs. If necessary, replace the path with the one leading to the log folder of your experiment

In [ ]:
%load_ext tensorboard
%tensorboard --logdir rl-models/ControlBurgersBench3/tensorboard-log

Now we can take a look at how the agent is performing:

In [ ]:
env = trainer.env

obs = env.reset()
bplt.burgers_figure('RL Reconstruction')
plt.plot(obs[0][:,0], color=bplt.gradient_color(0, step_count+1), linewidth=0.8)
plt.legend(['Initial state in dark red, final state in dark blue,'])
plt.ylim(-2, 2)
for frame in range(1, step_count):
    act = trainer.predict(obs, deterministic=True)
    obs, _, _, _ = env.step(act)
    plt.plot(obs[0][:,0], color=bplt.gradient_color(frame, step_count+1), linewidth=0.8)
plt.plot(env.goal_state.velocity.data[0,:,0])